In [36]:
import pandas as pd
import numpy as np
import seaborn as sns

In [37]:
train_data = pd.read_csv("./train.csv")
test_data = pd. read_csv("./test.csv")


In [38]:
def make_frame_ready(source_frame: pd.DataFrame) -> pd.DataFrame:
    

    #make new column group from the passengerId
    source_frame["Group"] = source_frame["PassengerId"].apply(lambda x: x.split("_")[0])

    #make new column family from the name of the passenger
    source_frame["Family"] = source_frame["Name"].apply(lambda x: str(x).split(" ")[-1])


    #impute missing family from group
    source_frame["Family"].isna = source_frame.groupby("Group")["Family"].agg(lambda x: x.value_counts().index[0] )

    #split cabin infor into three parts

    #Put cabin number into bins


    #create age category


    #set spending for cryosleepers


    #create totalSpending column


    #set spending for all children to 0


    #impute VIP status    
    
    return source_frame

In [39]:
train_data = make_frame_ready(train_data)